In [6]:
import numpy as np
import torch
import os

In [7]:
# Load GT labels
self_data_path = '/home/peisheng/3detr/scannet_data/scannet_all'
scan_name = 'scene0191_00'
instance_bboxes = np.load(os.path.join(self_data_path, scan_name) + "_bbox.npy")

In [8]:
instance_bboxes

array([[ 1.94911337,  2.54922485,  1.07194901,  0.17460835,  1.1070751 ,
         1.39844286,  8.        ],
       [ 1.06101453, -0.36034763,  0.43245816,  1.69858456,  0.86210084,
         0.87308061,  7.        ],
       [ 1.98960567,  0.49231637,  0.89994586,  0.31689119,  0.87098539,
         1.00589061,  9.        ],
       [ 0.21108061, -0.36262605,  0.21233006,  0.22650719,  0.32937795,
         0.45839486, 39.        ],
       [ 1.27606463, -0.55551457,  0.26375592,  0.64525551,  0.43832135,
         0.47023377,  5.        ],
       [ 1.97557068, -1.19884264,  0.8656804 ,  0.26435578,  0.67966241,
         1.12072575,  9.        ],
       [ 2.03211308, -2.61495972,  0.35489422,  0.22199845,  0.49363828,
         0.15295288,  9.        ]])

In [23]:
# Convert pseudo label to instance bbox. This is verified to be correct.
def pseudo_label_to_instance_bbox(pseudo_label):
    # pseudo_label is a list of length 3. The first element is the class index.
    # The second element is an array of (8, 3), which is the x,y,z coordinates of 8 corners of the bounding box.
    # The third element is the probality of the class.

    # PL to BB: 0 -> 0, -1 -> 2, 2 -> 1 
    instance_bbox = np.zeros((7, ))
    instance_bbox[0] = pseudo_label[1][:, 0].mean(axis=0)
    # swap instance_bbox[1] and instance_bbox[2], and time -1 to instance_bbox[2]
    # to convert from SDCoT format to 3DETR format
    instance_bbox[1] = pseudo_label[1][:, 2].mean(axis=0)
    instance_bbox[2] = pseudo_label[1][:, 1].mean(axis=0) * -1

    # dx is the same, but dy and dz are swapped
    instance_bbox[3] = pseudo_label[1][:, 0].max(
        axis=0) - pseudo_label[1][:, 0].min(axis=0)
    instance_bbox[4] = pseudo_label[1][:, 2].max(
        axis=0) - pseudo_label[1][:, 2].min(axis=0)
    instance_bbox[5] = pseudo_label[1][:, 1].max(
        axis=0) - pseudo_label[1][:, 1].min(axis=0)
    
    instance_bbox[6] = pseudo_label[0]
    return instance_bbox

In [24]:
# Convert instance_bboxes to pseudo labels
def instance_box_to_pseudo_label(instance_box):
    # instance_box is a numpy array of shape (7, )
    # The first element is the x coordinate of the center of the bounding box.
    # The second element is the y coordinate of the center of the bounding box.
    # The third element is the z coordinate of the center of the bounding box.
    # The fourth element is the x-axis length of the bounding box. (full length not half length)
    # The fifth element is the y-axis length of the bounding box. (full length not half length)
    # The sixth element is the z-axis length of the bounding box. (full length not half length)
    # The seventh element is the class index.

    # BB to PL: 0 -> 0, -2 -> 1, 1 -> 2
    pseudo_label = []
    pseudo_label.append(instance_box[6])
    pseudo_label.append(np.zeros((8, 3)))
    # pseudo_label[1][:, 0] is the x coordinates of the 8 corners of the bounding box. i.e., x0 and x1.
    pseudo_label[1][:, 0] = np.array([instance_box[0] - instance_box[3] / 2, instance_box[0] + instance_box[3] / 2, instance_box[0] + instance_box[3] / 2, instance_box[0] - instance_box[3] / 2, 
                                      instance_box[0] - instance_box[3] / 2, instance_box[0] + instance_box[3] / 2, instance_box[0] + instance_box[3] / 2, instance_box[0] - instance_box[3] / 2])
    # swap instance_box[1] and instance_box[2], and time -1 to instance_box[2]

    pseudo_label[1][:, 1] = -1 * np.array([instance_box[2] - instance_box[5] / 2, instance_box[2] - instance_box[5] / 2, instance_box[2] + instance_box[5] / 2, instance_box[2] + instance_box[5] / 2, 
                                      instance_box[2] - instance_box[5] / 2, instance_box[2] - instance_box[5] / 2, instance_box[2] + instance_box[5] / 2, instance_box[2] + instance_box[5] / 2])
    pseudo_label[1][:, 2] = np.array([(instance_box[1] - instance_box[4] / 2), instance_box[1] - instance_box[4] / 2, instance_box[1] - instance_box[4] / 2, instance_box[1] - instance_box[4] / 2, 
                                      instance_box[1] + instance_box[4] / 2, instance_box[1] + instance_box[4] / 2, instance_box[1] + instance_box[4] / 2, instance_box[1] + instance_box[4] / 2])
    pseudo_label.append(1) # probability of the class is 1 for gound truth

    return pseudo_label

In [25]:
pseudo_label = instance_box_to_pseudo_label(instance_bboxes[0])
print(pseudo_label)

[8.0, array([[ 1.86180919, -0.37272757,  1.99568731],
       [ 2.03641754, -0.37272757,  1.99568731],
       [ 2.03641754, -1.77117044,  1.99568731],
       [ 1.86180919, -1.77117044,  1.99568731],
       [ 1.86180919, -0.37272757,  3.1027624 ],
       [ 2.03641754, -0.37272757,  3.1027624 ],
       [ 2.03641754, -1.77117044,  3.1027624 ],
       [ 1.86180919, -1.77117044,  3.1027624 ]]), 1]


In [26]:
instance_box_from_pseudo_label = pseudo_label_to_instance_bbox(pseudo_label)
print(instance_box_from_pseudo_label)

[1.94911337 2.54922485 1.07194901 0.17460835 1.1070751  1.39844286
 8.        ]


In [27]:
print(instance_bboxes[0])

[1.94911337 2.54922485 1.07194901 0.17460835 1.1070751  1.39844286
 8.        ]


In [28]:
# check if the conversion is correct
print(np.allclose(instance_box_from_pseudo_label, instance_bboxes[0]))

True
